In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/MIB_lab/mydataset.py .

In [ ]:
import os
import numpy as np
import pandas as pd

from mydataset import myDataset

#torch
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

#torchvision
import torchvision
import torchvision.transforms as transforms
from torchvision import datasets
from torchvision.transforms import ToTensor
from torchvision.io import read_image

#plot
import matplotlib.pyplot as plt


In [ ]:
#find path to train data and test data (images)
data_dir = "/content/drive/MyDrive/MIB_lab/images"
train_dir = os.path.join(data_dir, 'train')
test_dir = os.path.join(data_dir, 'test')
# metadata array:
csv_path1 = os.path.join(train_dir,'train.csv')
csv_path2 = os.path.join(test_dir,'test.csv')
## Metadata dataframe ##
metadata1 = pd.read_csv( csv_path1, header=None )
metadata2 = pd.read_csv( csv_path2, header=None )

In [ ]:
# define transform used
transform = transforms.Compose(
    [#transforms.ToTensor(), 
     #output of torchvision dataset are PILImage range [0,1], ToTensor() convert this to Tensors
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     transforms.Scale((150,150))
     #transforms.ToTensor()
     ])

# retrieve the training and testing data from info obtained in csv files
training_data = myDataset(metadata=metadata1,img_dir=train_dir,transform=transform)
testing_data = myDataset(metadata=metadata2,img_dir=test_dir,transform=transform)

# load the data with dataloader
train_dataloader = DataLoader(training_data,batch_size=64,shuffle=True)
test_dataloader = DataLoader(testing_data,batch_size=64,shuffle=False)

/usr/local/lib/python3.7/dist-packages/torchvision/transforms/transforms.py:310: UserWarning: The use of the transforms.Scale transform is deprecated, please use transforms.Resize instead.
  warnings.warn("The use of the transforms.Scale transform is deprecated, " +


In [ ]:
#import torch.nn and functional which contains all functions necessary to define a convolutional neural network
import torch.nn as nn
import torch.nn.functional as F

# architecture construction
class Net(nn.Module): 
    def __init__(self): 
        super().__init__()
        # 4 convolutional layers
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=6,kernel_size=3,padding = 1) 
        self.conv2 = nn.Conv2d(in_channels=6,out_channels=12,kernel_size=3,padding = 1) 
        self.conv3 = nn.Conv2d(in_channels=12,out_channels=24,kernel_size=3,padding = 1) 

        # max pooling within 2x2
        self.pool = nn.MaxPool2d(stride=2,kernel_size=2)

        #3 linear layers
        # 3,150,150->6,150,150->6,75,75->12,75,75->12,37,37
        # ->24,37,37->24,18,18
        self.fc1 = nn.Linear(24*18*18, 512)
        self.fc2 = nn.Linear(512, 2)

    def forward(self, x): #override forward function
        # apply 1st conv layer, activate with relu
        x = self.pool(F.relu(self.conv1(x)))
        # apply 2nd, 3rd, and 4th conv layers, activate with relu
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))

        # flatten all dimensions except batch to perform linear layers
        x = torch.flatten(x, 1)
        # apply linear layers, activate with relu
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        
        return x

# initialize a net object
net = Net()

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss() # use Cross-entropy loss
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9) # use SGD with momentum for optimizer

In [ ]:
# training
# train for 2 epochs
for epoch in range(2): 

    # initiate running_loss
    running_loss = 0.0 

    for ith_batch,batch_data in enumerate(train_dataloader): 
        # obtain the images and labels
        img_batch,labels_batch = batch_data['image'],batch_data['label']

        # zero the parameter gradients (necessary because .backward() 
        # accumulate gradient for each parameter after each iteration)
        optimizer.zero_grad()

        # forward + backward + optimize
        # feed the img_batch (input) into the network
        outputs = net(img_batch)
        # calculate the cross-entropy loss
        loss = criterion(outputs, labels_batch)
        # backward
        loss.backward()
        # perform parameter update based on current gradient (stored in .grad) 
        # and update rule of SGD
        optimizer.step()

        # print statistics
        running_loss += loss.item() # .item() extracts loss values as floats
        
        # print every 10 mini-batches
        if ith_batch % 10 == 9:
            print('[%d, %5d] loss: %.3f' %
                  (epoch+1, ith_batch+1, running_loss/10))
            running_loss = 0.0

print('Finished Training')

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


[1,    10] loss: 0.688
[1,    20] loss: 0.667
[1,    30] loss: 0.652
[1,    40] loss: 0.630
[1,    50] loss: 0.604
[1,    60] loss: 0.560
[1,    70] loss: 0.502
[2,    10] loss: 0.436
[2,    20] loss: 0.385
[2,    30] loss: 0.343
[2,    40] loss: 0.287
[2,    50] loss: 0.268
[2,    60] loss: 0.259
[2,    70] loss: 0.244
Finished Training


In [ ]:
correct = 0
total = 0
# since we're not training, we don't need to calculate the gradients for our outputs
with torch.no_grad():
    for data in test_dataloader: # iterate through the data
        images, labels = data['image'],data['label']
        # calculate outputs by running images through the network 
        outputs = net(images)
        # the class with the highest energy is what we choose as prediction
        _, predicted = torch.max(outputs.data, 1)
        # increment total and correct
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
# print the accuracy
print('Accuracy of the networks: %d %%' % (
    100 * correct / total))

Accuracy of the networks: 90 %
